In [1]:
import json
import time
from pathlib import Path

import numpy as np
import qibo
from qibo import hamiltonians, set_backend
from qibo.models.dbi.double_bracket import (
    DoubleBracketGeneratorType,
    DoubleBracketIteration,
)

from boostvqe.ansatze import VQE, build_circuit
from boostvqe.utils import apply_dbi_steps, rotate_h_with_vqe

qibo.set_backend("numpy")

# set the path string which define the results
path = "../results/vqe_data/with_params/10q3l/sgd_10q_3l_42/"

# set the target epoch to which apply DBQA and the number of steps
target_epoch = 2000
dbi_steps = 1

# upload system configuration and parameters for all the training
with open(path + "optimization_results.json") as file:
    config = json.load(file)

losses = dict(np.load(path + "energies.npz"))["0"]
params = np.load(path + f"parameters/params_ite{target_epoch}.npy")


# build circuit, hamiltonian and VQE
hamiltonian = hamiltonians.XXZ(nqubits=config["nqubits"], delta=0.5)
circuit = build_circuit(config["nqubits"], config["nlayers"], "numpy")
vqe = VQE(circuit, hamiltonian)
zero_state = hamiltonian.backend.zero_state(config["nqubits"])
zero_state_t = np.transpose([zero_state])
target_energy = np.min(hamiltonian.eigenvalues())


# set target parameters into the VQE
vqe.circuit.set_parameters(params)
vqe_state = vqe.circuit().state()

ene1 = hamiltonian.expectation(vqe_state)


[Qibo 0.2.9|INFO|2024-06-06 13:44:32]: Using numpy backend on /CPU:0
INFO:qibo.config:Using numpy backend on /CPU:0
[Qibo 0.2.9|INFO|2024-06-06 13:44:33]: Using numpy backend on /CPU:0
INFO:qibo.config:Using numpy backend on /CPU:0


In the VQE numpy


In [3]:
v_vqe = vqe.circuit.unitary()

In [2]:
new_hamiltonian_matrix = rotate_h_with_vqe(hamiltonian=hamiltonian, vqe=vqe)
new_hamiltonian = hamiltonians.Hamiltonian(
    config["nqubits"], matrix=new_hamiltonian_matrix
)

np.linalg.norm(hamiltonian.matrix-new_hamiltonian.matrix)

0.0

In [15]:
h_vqe = v_vqe.conj().T @ hamiltonian.matrix @ v_vqe
h_vqe_ham = hamiltonians.Hamiltonian(nqubits = config["nqubits"],matrix=h_vqe)

In [5]:

np.linalg.norm(hamiltonian.matrix-h_vqe)

224.18868358763476

In [6]:
from qibo import symbols, hamiltonians
d = hamiltonians.SymbolicHamiltonian( sum([b*symbols.Z(j) for j,b in zip(range(config["nqubits"]),np.linspace(-1,1,config["nqubits"]))]))

-14.560174594055583


In [16]:
print(h_vqe_ham.expectation(zero_state_t))

-14.560174594055578


In [17]:
target_energy

-15.276131122065795

In [27]:
dm = d.dense.matrix

[Qibo 0.2.9|WARNING|2024-06-06 13:56:47]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


In [21]:
from copy import deepcopy
dbi = DoubleBracketIteration(
    hamiltonian=deepcopy(h_vqe_ham),
    mode=DoubleBracketGeneratorType.single_commutator,
)


In [25]:
dbi.h.expectation(zero_state_t)

-14.560174594055578

In [26]:

for t in [0.01]:
    dbi(t)
    print(dbi.h.expectation(zero_state_t))
    dbi.h = deepcopy(h_vqe_ham)

-14.782805005074469


In [ ]:
def u_gcr(h,d,t):
    ud =  d.exp(t)
    u = h.exp(t)
    return     ud.conj().T @ u @ud


In [32]:
h_vqe_ham.expectation(u_gcr(h_vqe_ham,d,np.sqrt(0.01))@zero_state_t)

-1.4827471249442334

In [ ]:
h_vqe_ham.expectation(u_gcr(h_vqe_ham,d,np.sqrt(0.01))@zero_state_t)

In [ ]:

# DBQA stuff
t0 = time.time()
print("Rotating with VQE")
new_hamiltonian_matrix = rotate_h_with_vqe(hamiltonian=hamiltonian, vqe=vqe)
new_hamiltonian = hamiltonians.Hamiltonian(
    config["nqubits"], matrix=new_hamiltonian_matrix
)
print(time.time() - t0)


In [ ]:
t = 0.01
s = np.sqrt(t)

In [33]:
target_energy

-15.276131122065795

In [ ]:
ene1

In [ ]:
from qibo import hamiltonians
import numpy as np
from boostvqe.compiling_XXZ import *

steps = 1
delta=0.5


h_xxz = hamiltonians.XXZ(nqubits=config["nqubits"], delta = delta)
u = h_xxz.exp(t)


In [ ]:
from qibo import symbols, hamiltonians
d = hamiltonians.SymbolicHamiltonian( sum([symbols.Z(j) for j in range(config["nqubits"])]))
circ_d = d.circuit(s)
ud = circ_d.unitary()

In [ ]:
from qibo import symbols, hamiltonians
dtest = hamiltonians.SymbolicHamiltonian( sum([symbols.Z(j) for j in range(config["nqubits"])])).dense
ud =  dtest.exp(-0.1)

In [ ]:
u2 = h_xxz.exp(t*np.sqrt(5))

In [ ]:
print(h_xxz.expectation(ud.conj().T@u2@ud@u2@u_gcr@ vqe_state))

In [ ]:
circ = nqubit_XXZ_decomposition(nqubits=config["nqubits"],t=t,delta=delta,steps=steps)
v = circ.unitary()



In [ ]:
u = v@v

In [ ]:
circ = nqubit_XXZ_decomposition(nqubits=config["nqubits"],t=t * np.sqrt(5),delta=delta,steps=steps)
v2 = circ.unitary()

In [ ]:
def try_to_speed_up_GC3( v,v2,d):
    return  (
    v@ v2.conj().T
    @ self.backend.calculate_matrix_exp(-step * (np.sqrt(5) - 1) / 2, d)
    @ v
    @ self.backend.calculate_matrix_exp(step * (np.sqrt(5) + 1) / 2, d)
    @ v.conj().T @  v.conj().T @v.conj().T @ v2.conj().T
    @ self.backend.calculate_matrix_exp(-step, d)
)


In [ ]:
from qibo import symbols, hamiltonians
d = hamiltonians.SymbolicHamiltonian( sum([symbols.Z(j) for j in range(config["nqubits"])])).dense

In [ ]:
from qibo.models.dbi.utils import *

pauli_operator_dict = generate_pauli_operator_dict(config["nqubits"], 1)
pauli_operators = list(pauli_operator_dict.values())
d_coef_pauli = np.ones(config["nqubits"])
d_coef_pauli = d_coef_pauli + np.random.rand(len(d_coef_pauli))
d_pauli = sum([d_coef_pauli[i] * pauli_operators[i] for i in range(len(d_coef_pauli))])


In [ ]:
np.linalg.norm(d_pauli - d)

In [ ]:
q = try_to_speed_up_GC3(v,v2,dbi.)

In [ ]:
def expose_gci3(self, step)
    return = (
    self.h.exp(-step * (np.sqrt(5) - 1) / 2)
    @ self.backend.calculate_matrix_exp(-step * (np.sqrt(5) - 1) / 2, d)
    @ self.h.exp(step)
    @ self.backend.calculate_matrix_exp(step * (np.sqrt(5) + 1) / 2, d)
    @ self.h.exp(-step * (3 - np.sqrt(5)) / 2)
    @ self.backend.calculate_matrix_exp(-step, d)
)



In [ ]:
u =expose_gci3(dbi,np.sqrt(t))


In [ ]:

dbi = DoubleBracketIteration(
    hamiltonian=new_hamiltonian,
    mode=DoubleBracketGeneratorType.group_commutator_3,
)

zero_state_t = np.transpose([zero_state])
energy_h0 = float(dbi.h.expectation(np.array(zero_state_t)))
fluctuations_h0 = float(dbi.h.energy_fluctuation(zero_state_t))


In [ ]:
dbi(0.1)

In [ ]:
d = dbi.diagonal_h_matrix
dbi.backend.calculate_matrix_exp(-0.1,d)

In [ ]:
dbi.backend.calculate_matrix_exp(-0.1,dbi.h.matrix)

In [ ]:
new_hamiltonian.exp(0.1)

In [ ]:
print(fluctuations_h0)
print(energy_h0)

In [ ]:
dbi.loss(0.01)

In [ ]:

print("Applying DBI steps")

print(time.time() - t0)
print(
    f"\nReached accuracy before DBI at iter {target_epoch}: {np.abs(target_energy - ene1)}"
)
print(
    f"Reached accuracy after DBI at iter {target_epoch}: {np.abs(target_energy - dbi_energies[-1])}"
)
print(
    f"Reached accuracy in the end of VQE long training: {np.abs(target_energy - losses[-1])}"
)
